# Random Forest usando TensorFlow

Implementación del algoritmo Random Forest utilizando la librería TensorFlow, aplicandolo a la clasificación de intrusos en redes. La base de datos puede ser consultada en ()

- Author: Ernesto Giron E.

In [8]:
from __future__ import print_function
from time import time
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from IPython.display import display, HTML

import tensorflow as tf
from tensorflow.contrib.tensor_forest.python import tensor_forest
# Ignoramos todos los GPUs, debido a que tf random forest no se beneficia de ello.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
# !export TF_CPP_MIN_LOG_LEVEL=2

In [9]:
# Cargamos los datos y seleccionamos algunas variables importantes
ataques_train = pd.read_csv('data/kddcup.data_clean.csv', sep=',', decimal='.')
print("Cantidad de observaciones %i con %i variables: " %(ataques_train.shape[0],ataques_train.shape[1]))
feature_cols = ['same_srv_rate', 'flag_SF', 'dst_host_same_srv_rate', 'service_private',
       'dst_host_srv_serror_rate', 'service_http', 'logged_in',
       'dst_host_srv_count', 'count', 'srv_serror_rate', 'flag_S0',
       'dst_host_serror_rate', 'dst_host_count', 'rerror_rate', 'serror_rate',
       'dst_host_rerror_rate', 'src_bytes', 'srv_rerror_rate',
       'dst_host_same_src_port_rate', 'dst_host_srv_rerror_rate',
       'protocol_type_udp', 'service_ecr_i', 'flag_REJ', 'service_pop_3',
       'protocol_type_tcp', 'diff_srv_rate', 'hot', 'dst_host_diff_srv_rate',
       'service_telnet', 'service_domain_u', 'wrong_fragment',
       'dst_host_srv_diff_host_rate', 'num_compromised', 'service_smtp',
       'srv_count', 'dst_bytes', 'srv_diff_host_rate', 'service_ftp_data',
       'duration', 'service_ftp', 'attack_category']

ataques_train = ataques_train[feature_cols]
# Cargamos los datos de validación del 10% de la competencia 
ataques_10prec_test = pd.read_csv('data/data_10per_test_preprocessed.csv', sep=',', decimal='.')
ataques_test = ataques_10prec_test[feature_cols]

print("Cantidad de observaciones %i con %i variables (Entrenamiento) " %(ataques_train.shape[0],ataques_train.shape[1]))
print("Cantidad de observaciones %i con %i variables (Validación) " %(ataques_test.shape[0],ataques_test.shape[1]))

# Balanceamos los datos
df_normal = ataques_train[ataques_train.attack_category=='normal']
df_dos = ataques_train[ataques_train.attack_category=='dos']
df_probe = ataques_train[ataques_train.attack_category=='probe']
df_r2l = ataques_train[ataques_train.attack_category=='r2l']
df_u2r = ataques_train[ataques_train.attack_category=='u2r']
#df_unknown = ataques_train[ataques_train.attack_category=='unknown']

# Remuestreo tomando solo un conjnto de datos menor en las clases de mayor frecuencia
df_normal_downsampled = resample(df_normal, replace=False, n_samples=200000, random_state=123)
df_dos_downsampled = resample(df_dos, replace=False, n_samples=100000, random_state=123)
# Combinar las clases con los nuevos datos remuestreados
ataques_train = pd.concat([df_normal_downsampled, df_dos_downsampled, df_probe, df_r2l, df_u2r])

# Eliminamos los datos "unknown" de la muestra de validación, los cuales no tenemos como entrenar
ataques_test = ataques_test[ataques_test.attack_category!='unknown']

# Mostrar las cantidades de los nuevos datos
print("Balanceo de datos: ")
print(ataques_train.attack_category.value_counts())
print(ataques_test.attack_category.value_counts())

# Definimos los datos en entrenamiento y validación
X = ataques_train.drop(['attack_category'], axis=1)
y = ataques_train.attack_category.copy()
X_test_40var = ataques_test.drop(['attack_category'], axis=1)
y_test_40var = ataques_test.attack_category.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=35)

# y_train debe ser númerico y entero para usar con RF
y_train_int = pd.factorize(y_train)
y_test_int = pd.factorize(y_test)
y_test_40var_int = pd.factorize(y_test_40var)

Cantidad de observaciones 1074992 con 122 variables: 
Cantidad de observaciones 1074992 con 41 variables (Entrenamiento) 
Cantidad de observaciones 311029 con 41 variables (Validación) 
Balanceo de datos: 
normal    200000
dos       100000
probe      13860
r2l          999
u2r           52
Name: attack_category, dtype: int64
dos       223298
normal     60593
r2l         5993
probe       2377
u2r           39
Name: attack_category, dtype: int64


In [3]:
# Parámetros del modelo
num_steps = 2000 # Número de pasos para entrenar
batch_size = 200 # Número de observaciones por lote
num_classes = len(y.unique().tolist()) # Los 5 categorias de tipos de ataques
num_features = X.shape[1] # Cantidad de atributos (40)
num_trees = 5  # Cantidad de árboles a sembrar
max_nodes = 50 # Cantidad de máxima de nodos

# Definimos X e Y para los datos de entrada y salida
X_ = tf.placeholder(tf.float32, shape=[None, num_features]) # None para luego agregar cuanta observación queramos
# Para random forest, las etiquetas deben ser de tipo entero (la clase id)
Y_ = tf.placeholder(tf.int32, shape=[None]) # Igual None por que depende del número de observaciones que tengamos

# Random Forest Parámetros
hparams = tensor_forest.ForestHParams(num_classes=num_classes,
                                      num_features=num_features,
                                      num_trees=num_trees,
                                      max_nodes=max_nodes).fill()


In [4]:
# Construimos nuestro modelo con Random Forest
# tf.reset_default_graph()
forest_graph = tensor_forest.RandomForestGraphs(hparams)
# Obtenemos los gráfos de entrenamiento y perdida
train_op = forest_graph.training_graph(X_, Y_)
loss_op = forest_graph.training_loss(X_, Y_)

# Medimos la precisión de nuestro modelo
infer_op = forest_graph.inference_graph(X_)
correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(Y_, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Inicializamos las variables a usar
init_vars = tf.global_variables_initializer()

INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'num_trees': 5, 'max_nodes': 50, 'bagging_fraction': 1.0, 'feature_bagging_fraction': 1.0, 'num_splits_to_consider': 40, 'max_fertile_nodes': 25, 'split_after_samples': 250, 'min_split_samples': 5, 'valid_leaf_threshold': 1, 'dominate_method': 'bootstrap', 'dominate_fraction': 0.99, 'num_classes': 5, 'num_features': 40, 'bagged_num_features': 40, 'bagged_features': None, 'regression': False, 'num_outputs': 1, 'num_output_columns': 6, 'split_initializations_per_input': 1, 'base_random_seed': 0}
INFO:tensorflow:training graph for tree: 0
INFO:tensorflow:training graph for tree: 1
INFO:tensorflow:training graph for tree: 2
INFO:tensorflow:training graph for tree: 3
INFO:tensorflow:training graph for tree: 4


In [5]:
def next_batch(num, data, labels, isDataFrame=True):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    if (isDataFrame):
        data_shuffle = [data.iloc[i] for i in idx]
        labels_shuffle = [labels[i] for i in idx]
    else:
        data_shuffle = [data[i] for i in idx]
        labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)


In [6]:
# Iniciamos una sesión de TensorFlow
sess = tf.Session()

# Ejecutamos el inicializador de variables
sess.run(init_vars)

# Entrenamos nuestro modelo
for i in range(1, num_steps + 1):
    # Preparamos los datos
    batch_x, batch_y = next_batch(batch_size, X_train, y_train_int[0])
    _, l = sess.run([train_op, loss_op], feed_dict={X_: batch_x, Y_: batch_y})
    if i % 50 == 0 or i == 1:
        acc = sess.run(accuracy_op, feed_dict={X_: batch_x, Y_: batch_y})
        print('Paso %i, Costo: %f, Precisión: %f' % (i, l, acc))

Paso 1, Costo: -0.000000, Precisión: 0.670000
Paso 50, Costo: -44.400002, Precisión: 0.955000
Paso 100, Costo: -48.000000, Precisión: 0.980000
Paso 150, Costo: -48.000000, Precisión: 0.985000
Paso 200, Costo: -48.000000, Precisión: 0.990000
Paso 250, Costo: -48.000000, Precisión: 0.975000
Paso 300, Costo: -48.000000, Precisión: 0.985000
Paso 350, Costo: -48.000000, Precisión: 0.970000
Paso 400, Costo: -48.000000, Precisión: 0.980000
Paso 450, Costo: -48.000000, Precisión: 0.980000
Paso 500, Costo: -48.000000, Precisión: 0.975000
Paso 550, Costo: -48.000000, Precisión: 0.985000
Paso 600, Costo: -48.000000, Precisión: 0.970000
Paso 650, Costo: -48.000000, Precisión: 0.980000
Paso 700, Costo: -48.000000, Precisión: 0.985000
Paso 750, Costo: -48.000000, Precisión: 0.990000
Paso 800, Costo: -48.000000, Precisión: 0.960000
Paso 850, Costo: -48.000000, Precisión: 0.990000
Paso 900, Costo: -48.000000, Precisión: 0.965000
Paso 950, Costo: -48.000000, Precisión: 0.995000
Paso 1000, Costo: -48.00

In [7]:
# Validar el Modelo
print("Precisión : ", sess.run(accuracy_op, feed_dict={X_: X_test, Y_: y_test_int[0]}))

# Validar el Modelo con los datos del concurso
print("Precisión TF Random Forest: ", sess.run(accuracy_op, feed_dict={X_: X_test_40var, Y_: y_test_40var_int[0]}))

Precisión :  0.0109236
Precisión TF Random Forest:  0.770103
